# Install and import dependencies

In [ ]:
!pip install -qU "ray[train]"==2.7.0 "ray[client]"==2.7.0 xgboost_ray scikit-learn

In [ ]:
from xgboost_ray import RayDMatrix, RayParams, train, predict
from sklearn.datasets import load_breast_cancer
import ray
import xgboost as xgb

# Initiate connection to the remote Ray cluster

In [ ]:
ray.init(address="ray://raycluster-kuberay-head-svc.default.svc.cluster.local:10001", runtime_env={"pip": ["xgboost", "xgboost_ray", "scikit-learn"]})

# Training

In [ ]:
@ray.remote
def train_model():
    train_x, train_y = load_breast_cancer(return_X_y=True)
    train_set = RayDMatrix(train_x, train_y)

    evals_result = {}
    bst = train(
        {
            "objective": "binary:logistic",
            "eval_metric": ["logloss", "error"],
        },
        train_set,
        evals_result=evals_result,
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=RayParams(
            num_actors=7,  # Number of remote actors
            cpus_per_actor=1))
    
    print("Final training error: {:.4f}".format(
        evals_result["train"]["error"][-1]))
    
    return bst

In [ ]:
model = ray.get(train_model.remote())
model.save_model("model.xgb")

# Batch Predictions

In [ ]:
data, labels = load_breast_cancer(return_X_y=True)

dpred = RayDMatrix(data, labels)

model = xgb.Booster(model_file="model.xgb")
pred_ray = predict(model, dpred, ray_params=RayParams(num_actors=2, cpus_per_actor=1))

print(pred_ray)

# Clean up

In [ ]:
ray.shutdown()